# NmapBuddy - Automated Port & Vulnerability Scanner


## 📚 Table of Contents
1. [Overview](#overview)
2. [Setup](#setup)
3. [Timestamp + Logging](#timestamp)
4. [Target Selection](#target)
5. [Nmap Scan Execution](#scan)
6. [Output & Results](#results)
7. [Future Features](#future)


# NmapBuddy - Automated Port and Vulnerability Scanner
---
**Created by:** Tamieka Horne  
**Date Started:** 4/27/2025  

This project automates port scanning and basic vulnerability detection using Nmap and Python.  
It is designed to streamline manual security assessments by allowing users to quickly identify open ports and potential vulnerabilities on a target system or network.


## 🛠️ Tools & Technologies

This project was built using:
- **Python 3.x**
- **Nmap** – for performing port and vulnerability scans
- **subprocess module** – to execute shell commands from within Python
- **datetime module** – for timestamping logs
- **Jupyter Notebook** – for interactive scripting, documentation, and analysis

These tools were chosen for their accessibility, effectiveness, and ability to support real-time iteration and automation in security assessments.


## 🔎 Project Summary

NmapBuddy automates the process of network scanning using Nmap and Python. Built in Jupyter Notebook, it helps identify open ports and basic vulnerabilities across one or more targets. Results can be saved in `.txt` or `.json` format and include logging functionality.

This project was developed to reinforce skills in automation, network reconnaissance, and Python scripting while streamlining repetitive manual scanning tasks.


In [21]:
# Install the python-nmap package
%pip install python-nmap

import nmap

# Initialize the scanner
scanner = nmap.PortScanner()

# Define target and ports
target = 'scanme.nmap.org'
ports = '20-100'

# Run a basic scan
print(f"Scanning {target} for ports {ports}...")

scanner.scan(target, ports)

# Display the scan results
for host in scanner.all_hosts():
    print(f"\nHost: {host} ({scanner[host].hostname()})")
    print(f"State: {scanner[host].state()}")

    for protocol in scanner[host].all_protocols():
        print(f"\nProtocol: {protocol}")
        ports = scanner[host][protocol].keys()
        for port in ports:
            print(f"Port: {port}\tState: {scanner[host][protocol][port]['state']}")


Note: you may need to restart the kernel to use updated packages.
Scanning scanme.nmap.org for ports 20-100...

Host: 45.33.32.156 (scanme.nmap.org)
State: up

Protocol: tcp
Port: 22	State: open
Port: 25	State: filtered
Port: 53	State: open
Port: 80	State: open


## Basic Nmap Scan
This cell runs a simple Nmap port scan on `scanme.nmap.org` from ports 20 to 100.
It prints the open ports and the status of the target host.


In [13]:
import os

# Create output folders
os.makedirs("output/json", exist_ok=True)
os.makedirs("output/txt", exist_ok=True)
os.makedirs("images", exist_ok=True)


In [20]:
import os
import json
from datetime import datetime
from colorama import Fore, Style, init
import nmap

init(autoreset=True)

# Create folders
os.makedirs("output/json", exist_ok=True)
os.makedirs("output/txt", exist_ok=True)
os.makedirs("images", exist_ok=True)

def save_as_text(scanner, host_key, original_target):
    file_path = f"output/txt/{original_target.replace('.', '_')}_scan.txt"
    with open(file_path, "w") as f:
        f.write(str(scanner[host_key]))
    print(Fore.GREEN + f"Saved to {file_path}")

def save_as_json(scanner, host_key, original_target):
    file_path = f"output/json/{original_target.replace('.', '_')}_scan.json"
    with open(file_path, "w") as f:
        json.dump(scanner[host_key], f, indent=2)
    print(Fore.GREEN + f"Saved to {file_path}")

def log_scan(target, fmt):
    with open("scan_log.txt", "a") as log:
        log.write(f"{datetime.now()} | {target} | {fmt.upper()} format\n")

def run_scan(target):
    print(Fore.CYAN + f"\nScanning {target} for ports 20-100...")
    nm = nmap.PortScanner()
    try:
        nm.scan(hosts=target, arguments="-p 20-100")
        return nm
    except Exception as e:
        print(Fore.RED + f"Scan error for {target}: {e}")
        return None

def menu():
    print(Style.BRIGHT + Fore.YELLOW + "===== nmapBuddy (Back to Basics) =====\n")
    raw_targets = input("Enter targets separated by commas (e.g., scanme.nmap.org,127.0.0.1): ").split(",")

    for raw in raw_targets:
        target = raw.strip()
        if not target or target.isdigit():
            print(Fore.RED + f"Skipping invalid target: {target}")
            continue

        scanner = run_scan(target)
        if not scanner or not scanner.all_hosts():
            print(Fore.RED + f"No results for {target}. Skipping.\n")
            continue

        host_key = scanner.all_hosts()[0]

        print(Fore.MAGENTA + f"\nChoose output format for {target}:\n1. Save as text\n2. Save as JSON")
        choice = input("Enter choice: ").strip()

        if choice == "1":
            save_as_text(scanner, host_key, target)
            log_scan(target, "text")
        elif choice == "2":
            save_as_json(scanner, host_key, target)
            log_scan(target, "json")
        else:
            print(Fore.RED + "Invalid choice. Skipping.\n")

menu()


===== nmapBuddy (Back to Basics) =====


Scanning scanme.nmap.org for ports 20-100...

Choose output format for scanme.nmap.org:
1. Save as text
2. Save as JSON
Saved to output/json/scanme_nmap_org_scan.json

Scanning 127.0.0.1 for ports 20-100...

Choose output format for 127.0.0.1:
1. Save as text
2. Save as JSON
Saved to output/json/127_0_0_1_scan.json
